## Setup globals
Fill in the `<user>`, `<password>`, and `<load-balancer-ip>` fields in the `MONGODB_URI` string.

In [1]:
ISLAND_IMAGE_NAME = "europe-west4-docker.pkg.dev/island-algorithm-infra/islands/island"
MONGODB_URI = "mongodb://<user>:<password>@<load-balancer-ip>:27017/prod" 

## Pod config

In [2]:
from dataclasses import dataclass, field


def fill_pod_manifest(
    image_name: str,
    pod_name: str,
    mongodb_uri: str,
    island_id: str,
    coupled_island_id: str,
) -> dict:
    pod_manifest = {
        "apiVersion": "v1",
        "kind": "Pod",
        "metadata": {"name": pod_name},
        "spec": {
            "containers": [
                {
                    "name": "island-container",
                    "image": image_name,
                    "env": [
                        {"name": "MONGODB_URI", "value": mongodb_uri},
                        {"name": "ISLAND_ID", "value": island_id},
                        {"name": "COUPLED_ISLAND_ID", "value": coupled_island_id},
                    ],
                }
            ],
        },
    }

    return pod_manifest


@dataclass
class Pod:
    id: int
    name: str
    namespace: str
    manifest: dict = field(default_factory=dict)


def create_pods(num_pods: int) -> tuple[Pod]:
    pods = []
    for pod_id in range(1, num_pods + 1):
        pod_name = f"island-pod-{pod_id}"
        island_id = f"ISLAND{pod_id}"
        coupled_island_id = f"ISLAND{pod_id - 1 if pod_id - 1 > 0 else num_pods}"
        manifest = fill_pod_manifest(
            image_name=ISLAND_IMAGE_NAME,
            pod_name=pod_name,
            mongodb_uri=MONGODB_URI,
            island_id=island_id,
            coupled_island_id=coupled_island_id,
        )
        pods.append(
            Pod(id=pod_id, name=pod_name, manifest=manifest, namespace="islands")
        )

    return tuple(pods)


PODS = create_pods(4)

## Algorithm Setup

In [3]:
from typing import Iterable
from kubernetes import client, config
from ipywidgets import Button, Layout
from IPython.display import display


def check_cluster() -> None:
    expected_cluster_name = "gke_island-algorithm-infra_europe-west4_islands"
    if config.kube_config.list_kube_config_contexts()[1]['name'] != expected_cluster_name:
        raise ValueError(f"Connected to the wrong cluster. Please connect to {expected_cluster_name}")


def deploy_pod(api_instance: client.CoreV1Api, pod: Pod) -> None:
    api_instance.create_namespaced_pod(body=pod.manifest, namespace=pod.namespace)
    print(f"Pod '{pod.name}' deployed.")


def delete_pod(api_instance: client.CoreV1Api, pod: Pod) -> None:
    try:
        api_instance.delete_namespaced_pod(name=pod.name, namespace=pod.namespace)
        print(f"Pod {pod.name} deleted")
    except client.exceptions.ApiException as e:
        print(f"Exception when calling CoreV1Api->delete_namespaced_pod: {e}")


def setup_islands(pods: Iterable[Pod]) -> None:
    config.load_kube_config()
    check_cluster()
    api_instance = client.CoreV1Api()

    for pod in pods:
        deploy_pod(api_instance, pod)


def teardown_islands(pods: Iterable[Pod]) -> None:
    config.load_kube_config()
    check_cluster()
    api_instance = client.CoreV1Api()

    for pod in pods:
        delete_pod(api_instance, pod)



pods_are_setup = False
setup_button = Button(description="Setup", layout=Layout(width="auto"))
teardown_button = Button(description="Teardown", layout=Layout(width="auto"), disabled=True)


def on_setup_button_clicked(button):
    global pods_are_setup
    if not pods_are_setup:
        setup_islands(PODS)  
        pods_are_setup = True
        setup_button.disabled = True
        teardown_button.disabled = False

def on_teardown_button_clicked(button):
    global pods_are_setup
    if pods_are_setup:
        teardown_islands(PODS)
        pods_are_setup = False
        setup_button.disabled = False
        teardown_button.disabled = True


setup_button.on_click(on_setup_button_clicked)
teardown_button.on_click(on_teardown_button_clicked)


display(setup_button)
display(teardown_button)

Button(description='Setup', layout=Layout(width='auto'), style=ButtonStyle())

Button(description='Teardown', disabled=True, layout=Layout(width='auto'), style=ButtonStyle())

Pod 'island-pod-1' deployed.
Pod 'island-pod-2' deployed.
Pod 'island-pod-3' deployed.
Pod 'island-pod-4' deployed.


## Monitoring Results

In [ ]:
from pymongo import MongoClient
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import Output, IntSlider
from IPython.display import clear_output, display
import time


output = Output()

def monitor_results(pods: Iterable[Pod]) -> None:
    client = MongoClient(MONGODB_URI)

    db = client["prod"]

    collection = db["solutions"]

    island_ids = [f"ISLAND{pod.id}" for pod in pods]

    while True:
        with output:
            clear_output(wait=True)
            fig, axs = plt.subplots(2, 2, figsize=(15, 10))
            fig.suptitle("Fitness over Time")

            for i, island_id in enumerate(island_ids):
                cursor = collection.find({"islandId": island_id})

                data_list = list(cursor)

                df = pd.DataFrame(data_list)

                df["timestamp"] = pd.to_datetime(df["timestamp"], format="%Y:%m:%d %H:%M:%S")

                df = df.sort_values(by="timestamp")

                axs[i // 2, i % 2].plot(df["timestamp"], df["Fitness"])
                axs[i // 2, i % 2].set_title("Island: {}".format(island_id))
                axs[i // 2, i % 2].set_xlabel("Time")
                axs[i // 2, i % 2].set_ylabel("Fitness")

            plt.tight_layout()
            plt.show()
            time.sleep(1)

display(output)
monitor_results(PODS)

Output()